In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['je suis desole je ne vous ai pas reconnues', 'i m sorry i didn t recognize you']


In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [9]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [11]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [12]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [15]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [17]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
1m 28s (- 22m 11s) (5 6%) 1.5582
2m 55s (- 20m 26s) (10 12%) 0.7082
4m 25s (- 19m 11s) (15 18%) 0.3793
5m 53s (- 17m 40s) (20 25%) 0.2160
7m 20s (- 16m 10s) (25 31%) 0.1356
8m 48s (- 14m 40s) (30 37%) 0.0952
10m 16s (- 13m 13s) (35 43%) 0.0724
11m 44s (- 11m 44s) (40 50%) 0.0582
13m 12s (- 10m 16s) (45 56%) 0.0498
14m 43s (- 8m 49s) (50 62%) 0.0451
16m 11s (- 7m 21s) (55 68%) 0.0402
18m 2s (- 6m 0s) (60 75%) 0.0371
19m 24s (- 4m 28s) (65 81%) 0.0352
20m 46s (- 2m 58s) (70 87%) 0.0333
22m 9s (- 1m 28s) (75 93%) 0.0319
23m 29s (- 0m 0s) (80 100%) 0.0302


In [18]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> il est la pour te proteger
= he s here to protect you
< he s here to protect you <EOS>

> il est plus fort que jamais
= he is stronger than ever
< he is stronger than ever <EOS>

> elle a tres peur des chiens
= she s very afraid of dogs
< she s very afraid of dogs <EOS>

> tu n es qu un etudiant
= you are nothing but a student
< you are nothing but a student <EOS>

> je suis un peu emeche
= i m a bit tipsy
< i m a bit tipsy <EOS>

> je suis la seule a avoir survecu
= i m the only one who survived
< i m the only one who survived <EOS>

> je suis beaucoup plus jeune que vous
= i m much younger than you
< i m much younger than you <EOS>

> il n est pas gentil avec elle
= he is not kind to her
< he is not kind to her <EOS>

> elle ne convient pas a la tache
= she isn t adequate to the task
< she isn t adequate to the task <EOS>

> il est competent en espagnol et en italien
= he is proficient in both spanish and italian
< he is proficient of our and sort of biology <EOS>



In [19]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')

input = il n est pas aussi grand que son pere
output = he is not as tall as his father <EOS>
input = je suis trop fatigue pour conduire
output = i m too tired to drive now <EOS>
input = je suis desole si c est une question idiote
output = i m sorry if this is a stupid question <EOS>
input = je suis reellement fiere de vous
output = i m really proud of you <EOS>


C:\Users\Matthew\AppData\Local\Temp\ipykernel_12364\1690937169.py:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
C:\Users\Matthew\AppData\Local\Temp\ipykernel_12364\1690937169.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
C:\Users\Matthew\AppData\Local\Temp\ipykernel_12364\1690937169.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Exercises

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, random_split
import gensim.downloader as api
from collections import Counter
import re

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
HIDDEN_SIZE = 512
EMBEDDING_DIM = 300  # GloVe dimension
NUM_LAYERS = 3
DROPOUT = 0.3
MAX_LENGTH = 25
LOAD_PRETRAINED = True

In [37]:
# 1. Load and preprocess data
class QADataset(Dataset):
    def __init__(self, parquet_path):
        self.df = pd.read_parquet(parquet_path)
        self.questions, self.answers = self.preprocess_data()
        
        # Build vocabularies
        self.question_vocab = self.build_vocab(self.questions)
        self.answer_vocab = self.build_vocab(self.answers)
        
        # Load pretrained embeddings using Gensim
        self.glove = api.load("glove-wiki-gigaword-300") if LOAD_PRETRAINED else None
        
    def preprocess_data(self):
        def normalize_text(text):
            text = text.lower().strip()
            text = re.sub(r"([.!?])", r" \1", text)
            text = re.sub(r"[^a-zA-Z.!?]+", r" ", text)
            return text
            
        questions = [normalize_text(q) for q in self.df['question']]
        answers = [normalize_text(a) for a in self.df['answer']]
        return questions, answers
    
    def build_vocab(self, sentences):
        word_counts = Counter()
        for sentence in sentences:
            word_counts.update(sentence.split())
        
        vocab = {
            'word2idx': {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3},
            'idx2word': {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>'}
        }
        
        idx = 4
        for word, count in word_counts.items():
            if count >= 2:  # Only keep words appearing at least twice
                vocab['word2idx'][word] = idx
                vocab['idx2word'][idx] = word
                idx += 1
                
        return vocab
    
    def sentence_to_tensor(self, sentence, vocab):
        indexes = [vocab['word2idx'].get(word, 3) for word in sentence.split()]
        indexes = [1] + indexes + [2]  # Add SOS and EOS
        return torch.tensor(indexes, dtype=torch.long)
    
    def create_emb_layer(self, vocab_size):
        emb_layer = nn.Embedding(vocab_size, EMBEDDING_DIM)
        if LOAD_PRETRAINED and self.glove is not None:
            for word, idx in self.question_vocab['word2idx'].items():
                if word in self.glove:
                    emb_layer.weight.data[idx] = torch.from_numpy(self.glove[word])
                else:
                    emb_layer.weight.data[idx] = torch.randn(EMBEDDING_DIM) * 0.25
        return emb_layer
    
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question_tensor = self.sentence_to_tensor(self.questions[idx], self.question_vocab)
        answer_tensor = self.sentence_to_tensor(self.answers[idx], self.answer_vocab)
        return question_tensor, answer_tensor


In [38]:
# 2. Enhanced Encoder with Pretrained Embeddings
class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.embedding = self.create_emb_layer(vocab_size)
        self.gru = nn.GRU(EMBEDDING_DIM, hidden_size, num_layers=num_layers,
                         dropout=dropout if num_layers>1 else 0, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, hidden_size)
        self.dropout = nn.Dropout(dropout)
        
    def create_emb_layer(self, vocab_size):
        emb_layer = nn.Embedding(vocab_size, EMBEDDING_DIM)
        if LOAD_PRETRAINED:
            weights = np.zeros((len(qa_dataset.question_vocab['word2idx']), EMBEDDING_DIM))
            for word, idx in qa_dataset.question_vocab['word2idx'].items():
                if word in qa_dataset.glove.stoi:
                    weights[idx] = qa_dataset.glove.vectors[qa_dataset.glove.stoi[word]]
                else:
                    weights[idx] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))
            emb_layer.load_state_dict({'weight': torch.from_numpy(weights)})
        return emb_layer
    
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.gru(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        return outputs, hidden

In [39]:
# 3. Enhanced Decoder with Attention
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, EMBEDDING_DIM)
        self.attention = nn.Linear(hidden_size + EMBEDDING_DIM, hidden_size)
        self.gru = nn.GRU(hidden_size + EMBEDDING_DIM, hidden_size, num_layers=num_layers,
                         dropout=dropout if num_layers>1 else 0)
        self.fc = nn.Linear(hidden_size*2, vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))
        attn_weights = F.softmax(self.attention(torch.cat((embedded, hidden), dim=1), dim=1))
        attn_applied = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs.permute(1,0,2))
        gru_input = torch.cat((embedded, attn_applied.squeeze(1)), dim=1)
        output, hidden = self.gru(gru_input.unsqueeze(0), hidden.unsqueeze(0))
        prediction = self.fc(torch.cat((output.squeeze(0), attn_applied.squeeze(1)), dim=1))
        return prediction, hidden.squeeze(0)

In [40]:
# 4. Training Setup
def collate_batch(batch):
    questions, answers = zip(*batch)
    questions_pad = pad_sequence(questions, padding_value=0)
    answers_pad = pad_sequence(answers, padding_value=0)
    return questions_pad.to(device), answers_pad.to(device)

In [45]:
# Initialize dataset and dataloader
qa_dataset = QADataset('Question_Answer.parquet')
train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [42]:
# Model initialization
encoder = Encoder(len(qa_dataset.question_vocab['word2idx']), HIDDEN_SIZE, NUM_LAYERS, DROPOUT).to(device)
decoder = Decoder(len(qa_dataset.answer_vocab['word2idx']), HIDDEN_SIZE, NUM_LAYERS, DROPOUT).to(device)

# Optimizer and loss
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [43]:
# 5. Training Loop with Gradient Clipping
def train(model, iterator, optimizer, criterion, clip):
    encoder, decoder = model
    encoder.train()
    decoder.train()
    epoch_loss = 0
    
    for src, trg in iterator:
        optimizer.zero_grad()
        encoder_outputs, hidden = encoder(src)
        decoder_input = trg[0,:]
        loss = 0
        
        for t in range(1, trg.shape[0]):
            predictions, hidden = decoder(decoder_input, hidden, encoder_outputs)
            loss += criterion(predictions, trg[t])
            decoder_input = trg[t] if random.random() < 0.5 else predictions.argmax(1)
            
        loss.backward()
        torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() / trg.shape[0]
        
    return epoch_loss / len(iterator)

In [44]:
# 6. Evaluation
def answer_question(question, encoder, decoder, dataset):
    encoder.eval()
    decoder.eval()
    
    with torch.no_grad():
        question_tensor = dataset.sentence_to_tensor(question, dataset.question_vocab).unsqueeze(1).to(device)
        encoder_outputs, hidden = encoder(question_tensor)
        
        answer = []
        decoder_input = torch.tensor([1], device=device)  # SOS token
        
        for _ in range(MAX_LENGTH):
            predictions, hidden = decoder(decoder_input, hidden, encoder_outputs)
            top_idx = predictions.argmax(1).item()
            
            if top_idx == 2:  # EOS token
                break
                
            answer.append(dataset.answer_vocab['idx2word'].get(top_idx, '<UNK>'))
            decoder_input = torch.tensor([top_idx], device=device)
            
        return ' '.join(answer)